In [ ]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [ ]:
df = pd.read_csv('BBGM_League_109_all_seasons_Average_Stats.csv')
df = df[df['MP']*df['G'] > 10]

In [ ]:
pm,ms = df['+/-'].mean(), df['+/-'].std()

In [ ]:
df.columns

In [ ]:
plt.hist(df['+/-'])

In [ ]:
# full
stats_to_est = [ 'FG', 'FGA', '3P', '3PA',  '2P', '2PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'TOV', 'STL', 'BLK', 'BA', 'PF', 'PTS']
# pruned by hand based on t-values, R^2 = .804
stats_to_est = [                    '3PA',        '2PA',        'FTA',              'TRB',       'AST', 'TOV', 'STL', 'BLK',       'PF', 'PTS']
#R^2 = .773 w/o +/-, 0.788 with
stats_to_est = [                                                                    'TRB',        'AST', 'TOV', 'STL', 'BLK',      'PF', 'PTS', '+/-']

sT = [_ +'p36' for _ in stats_to_est]


In [ ]:
p_36_stats = []
for s in stats_to_est:
    df[s + 'p36'] = 36*df[s]/np.maximum(df['MP'],1)
    p_36_stats.append(s+'p36')
p_36_stats = p_36_stats #+['+/-']#+ ['TS%']

In [ ]:
res = sm.WLS(df['Ovr'],sm.add_constant(df[p_36_stats]),df['MP']*df['G']).fit()
res.summary() # -6.7306e+05 1.346e+06 1.005e+05

In [ ]:
import sklearn.linear_model as linear_model
clf = linear_model.RidgeCV(np.logspace(-3,3,101))
p = 1
clf.fit(df[p_36_stats],(df['Ovr'])**p,df['MP']*df['G'])
df['sOVR'] = np.maximum(0,clf.predict(df[p_36_stats]))**(1/p)
plt.scatter(df['Ovr'],df['sOVR'],s=5,alpha=0.5)


In [ ]:
plt.scatter(df['Ovr'],res.predict(),s=5,alpha=0.5)

In [ ]:
df['sOVR'] = clf.predict(df[p_36_stats])**(1/p)#res.predict()
df['PS'] = df['Ovr']/df['sOVR']

In [ ]:
plt.style.use('fivethirtyeight')

In [ ]:
plt.plot(df[['Age','PS','Ovr','sOVR']].groupby('Age').mean()['PS'])
plt.xlim(20,44)
plt.ylabel('Ovr ÷ sOVR')
plt.xlabel('Age')

In [ ]:
plt.hist(dfs.Age,13,density='True')
plt.xlim(20,44)
plt.xlabel('Age')
plt.ylabel('Density')

In [ ]:
res.params

In [ ]:
dft = df.groupby(['pid','Age']).mean()

In [ ]:
from collections import defaultdict
p_o = defaultdict(lambda: defaultdict(list))
for row in dft[['Pot','Ovr','sOVR']].itertuples():
    p_o[row[0][0]][row[0][1]].append(row[1:])

In [ ]:
p_y = defaultdict(list)
for p,yd in p_o.items():
    for y,s in yd.items():
        if y+1 in yd and y+2 in yd and y+3 in yd:
            X = list(s[0][:-1])
            Y = np.mean([yd[y+i][-1] for i in range(4)])
            p_y[y].append(X + [Y])

In [ ]:
print('5 year value by age\nage scale [pot ovr]')
sA = []
sB = []
for age in sorted(p_y.keys()):
    Xy = np.array(p_y[age])
    if (Xy[:,0] == Xy[:,1]).all():
        Xy = Xy[:,1:]
    res2 = sm.OLS(Xy[:,-1],Xy[:,:-1]).fit()
    v = res2.params
    vS = sum(v)
    sA.append((age,vS))
    sB.append((age,v[0]/vS))
    print(age,np.round(vS,2),np.round(v/vS,2))

In [ ]:
sA = np.array(sA)
plt.plot(sA[:,0],sA[:,1])
plt.xlim(20,40)
plt.xlabel('age')
plt.ylabel('multi-year value')
plt.title('4 year [reg tree]')

In [ ]:
sB = np.array(sB)
plt.plot(sB[:,0],sB[:,1])
plt.xlim(20,28)
plt.xlabel('age')
plt.ylabel('potential weight')
plt.title('4 year [reg tree]')